https://jira.x5.ru/browse/CVMUPG-812

In [1]:
name = 'CVMUPG-812_friskies'

In [2]:
import os
import re
from pyspark.sql import SparkSession
from typing import List, Dict, Callable
import socket

spark = None

EXECUTOR_ENV = 'hdfs:///share/products/cvm5/lib/python/anaconda_2.4.4_ds.tar.gz'  # 2.4.4 
SPARK_ARCHIVE = 'hdfs:///share/lib/spark/sparkjars-2.4.4.zip'                     # 2.4.4
#EXECUTOR_ENV = 'hdfs:///share/lib/python/env/anaconda-2019.07.tar.gz'
#SPARK_ARCHIVE = 'hdfs:///share/lib/spark/sparkjars-2.3.1.zip'

os.environ["ARROW_LIBHDFS_DIR"] = "/usr/hdp/2.6.5.0-292/usr/lib"
os.environ['HADOOP_HOME'] = '/usr/hdp/current/hadoop-client/'
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64/'
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf/'
os.environ['SPARK_HOME'] = '/opt/conda/lib/python3.7/site-packages/pyspark'
os.environ['PYSPARK_PYTHON'] = 'anaconda_2.4.4_ds.tar.gz/bin/python3'             # 2.4.4
#os.environ['PYSPARK_PYTHON'] = 'anaconda-2019.07.tar.gz/bin/python3'






def restart_spark(task_name: str, num_executors: int, executor_memory='4G', executor_cores=2,
                  driver_memory='2G', queue='cvm5-rnd', additional_params: Dict[str, str] = None):
    global spark

    if spark:
        sc = spark.sparkContext
        if sc and sc._jsc:
            if not sc._jsc.sc().isStopped():
                print('Using cached spark')
                return sc, spark

    need_ports_for_app = 3
    user_tcp_ports = _get_user_tcp_ports()
    free_ports = _get_free_ports(user_tcp_ports)
    assert len(free_ports) >= need_ports_for_app, \
        f"Not enough free ports ({len(free_ports)}), need {need_ports_for_app}, stop other apps"
    app_ports = free_ports[:need_ports_for_app]

    host_ip = os.getenv('HOST_IP')
    
    spark_session = (
        SparkSession
        .builder
        .appName(task_name)
        .master('yarn')
        .config('spark.driver.memory', driver_memory)
        .config('spark.driver.maxResultSize', driver_memory)
        .config('spark.executor.cores', executor_cores)
        .config('spark.executor.memory', executor_memory)
        .config('spark.executor.memoryOverhead', '1G')
        .config('spark.dynamicAllocation.enabled', 'true')
        .config('spark.dynamicAllocation.maxExecutors', num_executors)
        .config('spark.sql.broadcastTimeout', '36000')
        .config('spark.dynamicAllocation.cachedExecutorIdleTimeout', '1200s')
        .config('spark.ui.port', app_ports[0])
        .config('spark.blockManager.port', app_ports[1])
        .config('spark.driver.port', app_ports[2])
        .config('spark.driver.host', host_ip)
        .config('spark.driver.bindAddress', '0.0.0.0')
        .config('spark.driver.extraLibraryPath', '/usr/hdp/2.6.5.0-292/hadoop/lib/native')
        .config('spark.driver.extraJavaOptions', '-Dhdp.version=current')
        .config('spark.debug.maxToStringFields', '50')
        .config('spark.yarn.queue', queue)
        .config('spark.yarn.dist.archives', EXECUTOR_ENV)
        .config('spark.yarn.archive', SPARK_ARCHIVE)
        .config('spark.yarn.am.extraJavaOptions', '-Dhdp.version=current')
        .config('spark.rpc.message.maxSize', '1024')
        .config('spark.sql.warehouse.dir', '/apps/hive/warehouse')
        .config('spark.sql.execution.pandas.respectSessionTimeZone', 'false')
        .config('spark.sql.orc.filterPushdown', 'true')
        .config('spark.sql.hive.convertMetastoreOrc', 'true')
        .config('spark.shuffle.service.enabled', 'true')
        .config('spark.hadoop.yarn.timeline-service.enabled', 'false')
        .config('spark.hadoop.yarn.client.failover-proxy-provider',
                'org.apache.hadoop.yarn.client.ConfiguredRMFailoverProxyProvider')
        .config('spark.serializer', 'org.apache.spark.serializer.KryoSerializer')
        .config('spark.kryoserializer.buffer.max', '128m')
        .config('spark.executor.extraLibraryPath', '/usr/hdp/2.6.5.0-292/hadoop/lib/native')
    )

    if additional_params:
        for key, value in additional_params.items():
            spark_session = spark_session.config(key, value)

    spark = (
        spark_session
        .enableHiveSupport()
        .getOrCreate()
    )
    sc = spark.sparkContext

    return sc, spark


def _get_user_tcp_ports() -> List[str]:
    regexp = re.compile(r'-2e')
    envuser= os.getenv('HOSTNAME')
    if regexp.search(envuser):       
      _, user_name, user_surname = envuser.upper().split('-')
      user_full_name = '_'.join([user_name, user_surname])
    else:
      _, user_name  = envuser.upper().split('-') 
      user_full_name = user_name
    user_tcp_ports = [v for k, v in os.environ.items() if user_full_name in k and k.endswith('TCP_PORT')]
    return user_tcp_ports


def _get_free_ports(ports: List[str]):
    free_ports = []
    for port in ports:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            if s.connect_ex(('0.0.0.0', int(port))) != 0:
                free_ports.append(port)
    return free_ports


In [3]:
sc, spark = restart_spark(
                    name, 
                    21, 
                    executor_memory='5G', 
                    executor_cores=3, 
                    driver_memory='7G', 
                    additional_params={"spark.sql.shuffle.partitions": "300"})

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/06 12:24:36 WARN cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!


In [4]:
sc.setLogLevel('ERROR')

In [5]:
import pandas as pd
import sys
from pyspark.sql import functions as F
import datetime
from datetime import timedelta

sys.path.append('/home/jovyan/glow-byte-filters-pyspark')
from logic_filters import * 
from segmentation import *

In [6]:
LOYALTY_CARDS = "hive_ssa_tc5.loyalty_card"
LOYALTY_CARDHOLDERS = "hive_ssa_tc5.loyalty_cardholder"
ACCOUNTS = "hive_ssa_tc5.account"
CVM5_GUESTS = "hive_cvm_acrm.cvm5_guests"

DIM_STORE = "hive_ssa_main.dim_store"
CHECKS_HEADERS = "hive_ssa_main.fct_rtl_txn"
CHECKS_ITEMS = "hive_ssa_main.fct_rtl_txn_item"
PRODUCTS = "hive_ssa_tc5.cvm_product"

### Проверка на доступность плю в магазинах

GEO - RF

In [8]:
# federal_subject = [22, 42, 54]
# macroregion_dk = ['MRDCT', 'MRDCN']

In [7]:
plu_codes = [4125613, 4125614]

In [8]:
pplu = spark.createDataFrame(pd.DataFrame({"plu_code": plu_codes}))

shops = (spark.table(DIM_STORE)
            .filter(F.col('valid_to_dttm')== datetime.datetime(5999, 1, 1, 0, 0))
#             .filter(F.col('federal_subject_dk').isin(federal_subject))
#             .filter(F.col('macroregion_dk').isin(macroregion_dk))
            .selectExpr('store_id as plant')
            .distinct()
            #.toPandas()['plant']
            #.tolist()
            )


tc5_stores_assort = (
    spark.table('HIVE_SSA_MAIN.ASSORTMENT_X_PLU_X_STORE')
    .withColumnRenamed('plu_id', 'plu_code')
    .withColumnRenamed('store_id', 'plant')
    .filter(F.col("plu_negate_flg") != 1)
    .selectExpr('plu_code', 'plant')
    .join(F.broadcast(pplu), 'plu_code', 'inner')
)

plants = (shops
                .join(tc5_stores_assort, on='plant', how='inner')
                .select('plant')
                .distinct()
                .toPandas()['plant']
                .tolist()
         )

len(plants)

[Stage 3:==>                                                    (23 + 33) / 423]21/12/06 12:34:14 ERROR cluster.YarnScheduler: Lost executor 2 on mn-hd0351.x5.ru: Unable to create executor due to Unable to register with external shuffle server due to : java.util.concurrent.TimeoutException: Timeout waiting for task.
[Stage 3:======================================================>(418 + 5) / 423]21/12/06 12:35:32 ERROR cluster.YarnScheduler: Lost executor 22 on mn-hd0197.x5.ru: Unable to create executor due to Unable to register with external shuffle server due to : java.util.concurrent.TimeoutException: Timeout waiting for task.
[Stage 4:=>                                                      (7 + 54) / 300]21/12/06 12:37:49 ERROR cluster.YarnScheduler: Lost executor 23 on mn-hd0014.x5.ru: Unable to create executor due to Unable to register with external shuffle server due to : java.util.concurrent.TimeoutException: Timeout waiting for task.
[Stage 4:=>                                 

14882

### Выбираем гостей нужного юзкейса

In [9]:
usecase = ['cross', 'upgrade', 'ump']
dt = datetime.date.today()
start_date = datetime.date(2021, 9, 1)
end_date = datetime.date(2021, 12, 1)

In [10]:
customers_usecase = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.col('calculation_dt') >= dt)
                     .filter(F.col('usecase').isin(usecase))
                     .select('account_no', 'customer_rk')
                            )

In [11]:
customers_usecase.count()

21/12/06 12:45:54 ERROR client.TransportResponseHandler: Still have 1 requests outstanding when connection from /192.168.234.68:40016 is closed


15941662

### Проверяем на доступность отобранных гостей в определенную дату

In [12]:
check_date = '2021-12-24'

In [13]:
guests = remove_global_control_group(spark, customers_usecase, check_date)

 Фильтр Глобальная контрольная группа - валиден.


[Stage 15:=====================================================>(297 + 3) / 300]

	 Отсёк 1 358 803 записей.


In [14]:
seg_sms1 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:09


In [15]:
seg_sms2 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[1], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:08


In [16]:
seg_sms3 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[2], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:19


In [17]:
seg_sms = seg_sms1.union(seg_sms2).union(seg_sms3).distinct()

In [18]:
seg_sms.write.parquet('temp812_1', mode='overwrite')
seg_sms = spark.read.parquet('temp812_1')

In [19]:
seg_sms.count()

5100546

# seg1

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [20]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [21]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [22]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
#                         .filter(F.col('gender_dk').isin(['F', 'U']))
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg_sms, on='customer_rk', how='inner')

In [23]:
checks_headers_tc5 = checks_headers_tc5.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [25]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                                ,'zsalnovat' #price
                               )
                   )


In [26]:
plu_hierarchy_lvl_4_dk = ['FD0703001']
# syntethic_category_id = [42]
# plu_brand_code = ['L473','2945','2657','3624','6390','6416']
plu_not_in = [4125613, 4125614]
# plu_id = [3680376, 3623999, 45105, 3680375, 3624002, 3623997, 3623998, 3624001, 4081735, 3680377, 3179905]

In [27]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
                 .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
#                  .filter(F.col('plu_brand_code').isin(plu_brand_code))
#                  .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
                 .filter(~F.col('plu_id').isin(plu_not_in))
#                  .filter(F.col('plu_id').isin(plu_id))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [28]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [29]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [32]:
accs = ['account_no']
pdf = checks_tc5.groupby(accs).agg(F.mean('zsalnovat').alias('price_mean'))
checks_tc5 = checks_tc5.join(pdf, on=accs)

In [33]:
seg1 = (checks_tc5.select('customer_rk', 'price_mean')
                  .distinct()
      )

In [34]:
seg1.write.parquet('temp812_2', mode='overwrite')
seg1 = spark.read.parquet('temp812_2')

[Stage 93:>(13 + 2) / 140][Stage 94:> (7 + 4) / 210][Stage 96:> (0 + 10) / 43]21/12/06 13:43:46 ERROR cluster.YarnScheduler: Lost executor 75 on mn-hd0321.x5.ru: Unable to create executor due to Unable to register with external shuffle server due to : java.util.concurrent.TimeoutException: Timeout waiting for task.


In [35]:
seg1.count()

1551883

In [36]:
seg_pd = seg1.toPandas()

In [37]:
len(seg_pd)

1551883

In [60]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [61]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [62]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
#                         .filter(F.col('gender_dk').isin(['F', 'U']))
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg_sms, on='customer_rk', how='inner')

In [63]:
checks_headers_tc5 = checks_headers_tc5.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [64]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               )
                   )


In [65]:
# plu_hierarchy_lvl_4_dk = ['FD0703001']
# syntethic_category_id = [42]
# plu_brand_code = ['L473','2945','2657','3624','6390','6416']
# plu_not_in = [4125613, 4125614]
plu_id = [4125613, 4125614]

In [66]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
#                  .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
#                  .filter(F.col('plu_brand_code').isin(plu_brand_code))
#                  .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .filter(F.col('plu_id').isin(plu_id))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [67]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [68]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [69]:
seg2 = (checks_tc5.select('customer_rk')
                  .distinct()
      )

In [70]:
seg2.write.parquet('temp812_3', mode='overwrite')
seg2 = spark.read.parquet('temp812_3')

In [71]:
seg2.count()

7618

In [ ]:
seg_frisc = seg2.toPandas()

In [81]:
seg_frisc.to_csv(name + '_' + 'use_frisc' + '_All_uc' + str(dt) + '.csv', index=False)

In [50]:
seg_pd_type_flt = seg_pd.copy()

In [51]:
seg_pd_type_flt['price_mean'] = seg_pd_type_flt['price_mean'].astype('float')

In [54]:
seg_pd_lower = seg_pd_type_flt[seg_pd_type_flt['price_mean'] <= seg_pd_type_flt['price_mean'].quantile(0.33)]

In [104]:
seg_pd_middle = seg_pd_type_flt[(seg_pd_type_flt['price_mean'] <= seg_pd_type_flt['price_mean'].quantile(0.66))
                               & (seg_pd_type_flt['price_mean'] > seg_pd_type_flt['price_mean'].quantile(0.33))]

In [80]:
seg_pd_middle.to_csv(name + '_' + 'middle' + '_All_uc' + str(dt) + '.csv', index=False)

In [123]:
print(len(seg_pd_middle), len(seg_pd_lower), len(seg_frisc))

512120 512124 7618


In [130]:
set_guest = set(seg_pd_middle['customer_rk'].tolist()) | set(seg_pd_lower['customer_rk'].tolist())

In [133]:
len(set_guest)

1024244

In [134]:
len(seg_frisc)

7618

In [135]:
set_guest.difference_update(set(seg_frisc['customer_rk'].tolist()))

In [139]:
len(set_guest)

1018256

In [57]:
seg_no = 'seg1'

In [79]:
seg_pd_lower.to_csv(name + '_' + 'lower' + '_All_uc' + str(dt) + '.csv', index=False)

In [58]:
seg_pd_lower.to_csv(name + '_' + 'lower' + '_All_uc' + str(dt) + '.csv', index=False)
seg_pd_lower = seg_pd_lower.astype('str')
cross, upgrade, ump = on_usecases(seg_pd_lower)

In [145]:
seg_guest_pd = pd.DataFrame(set_guest, columns=['customer_rk'])
seg_guest_pd.to_csv(name + '_' + seg_no + '_All_uc' + str(dt) + '.csv', index=False)
cross, upgrade, ump = on_usecases(seg_guest_pd)

In [148]:
len(seg_guest_pd)

1018256

In [147]:
cross.to_csv(name + '_' + seg_no + '_' + usecase[0] + '_' + str(dt) + '.csv', index=False)
upgrade.to_csv(name + '_' + seg_no + '_' + usecase[1] + '_' + str(dt) + '.csv', index=False)
ump.to_csv(name + '_' + seg_no + '_' + usecase[2] + '_' + str(dt) + '.csv', index=False)

In [146]:
print('cross: {}, upgrade: {}, ump: {}'.format(len(cross), len(upgrade), len(ump)))

cross: 556994, upgrade: 203018, ump: 258244
